### libraries import

In [2]:
import os

#gradio interface
import gradio as gr

from transformers import AutoModelForCausalLM,AutoTokenizer
import torch

#STT (speech to text)
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa

#TTS (text to speech)
import torch

#regular expressions
import re

#langchain and function calling
from typing import List, Literal, Union
import requests
from functools import partial
import math


#langchain, not used anymore since I had to find another way fast to stop using the endpoint, but could be interesting to reuse 
from langchain.tools.base import StructuredTool
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain.prompts import StringPromptTemplate



from datetime import datetime, timedelta, timezone
from transformers import pipeline
import inspect

/opt/homebrew/Caskroom/miniconda/base/envs/llm/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniconda/base/envs/llm/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniconda/base/envs/llm/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
from langchain.memory import ConversationSummaryBufferMemory

In [4]:
# load api key from .env file
# weather api and tomtom api key
from dotenv import load_dotenv
load_dotenv()
WHEATHER_API_KEY = os.getenv("WEATHER_API_KEY")
TOMTOM_KEY = os.getenv("TOMTOM_API_KEY")

In [5]:
from apis import *

### Models loads

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
from langchain_community.llms import Ollama
# https://github.com/ollama/ollama/blob/main/docs/api.md

llm = Ollama(model="nexusraven")

In [8]:
from skills import execute_function_call

In [9]:
execute_function_call("Call: get_weather(location='49.611622,6.131935') ", dry_run=True)

get_weather(**{'location': '49.611622,6.131935'})


'Dry run successful'

## Function calling with NexusRaven 

In [10]:
import skills
from skills import get_weather, get_forecast, find_route

In [21]:
print(skills.SKILLS_PROMPT)

OPTION:
<func_start>get_weather(location: str = '', **kwargs)<func_end>
<docstring_start>
Returns the CURRENT weather in a specified location.
Args:
location (string) : Required. The name of the location, could be a city or lat/longitude in the following format latitude,longitude (example: 37.7749,-122.4194).
<docstring_end>
OPTION:
<func_start>get_forecast(city_name: str = '', when=0, **kwargs)<func_end>
<docstring_start>
Returns the weather forecast in a specified number of days for a specified city .
Args:
city_name (string) : Required. The name of the city.
when (int) : Required. in number of days (until the day for which we want to know the forecast) (example: tomorrow is 1, in two days is 2, etc.)
<docstring_end>
OPTION:
<func_start>find_route(lat_depart='0', lon_depart='0', city_depart='', address_destination='', depart_time='', **kwargs)<func_end>
<docstring_start>
Return the distance and the estimated time to go to a specific destination from the current place, at a specified 

In [22]:
# Moved to skills/__init__.py here only for experimentation
def format_functions_for_prompt_raven(*functions):
    formatted_functions = []
    for func in functions:
        signature = f"{func.__name__}{inspect.signature(func)}"
        docstring = inspect.getdoc(func)
        formatted_functions.append(
            f"OPTION:\n<func_start>{signature}<func_end>\n<docstring_start>\n{docstring}\n<docstring_end>"
        )
    return "\n".join(formatted_functions)

In [23]:
formatted_prompt = format_functions_for_prompt_raven(get_weather, find_points_of_interest, find_route, get_forecast, search_along_route)

In [24]:
print(formatted_prompt)

OPTION:
<func_start>get_weather(location: str = '', **kwargs)<func_end>
<docstring_start>
Returns the CURRENT weather in a specified location.
Args:
location (string) : Required. The name of the location, could be a city or lat/longitude in the following format latitude,longitude (example: 37.7749,-122.4194).
<docstring_end>
OPTION:
<func_start>find_points_of_interest(lat='0', lon='0', city='', type_of_poi='restaurant', **kwargs)<func_end>
<docstring_start>
Return some of the closest points of interest for a specific location and type of point of interest. The more parameters there are, the more precise.
:param lat (string):  latitude
:param lon (string):  longitude
:param city (string): Required. city
:param type_of_poi (string): Required. type of point of interest depending on what the user wants to do.
<docstring_end>
OPTION:
<func_start>find_route(lat_depart='0', lon_depart='0', city_depart='', address_destination='', depart_time='', **kwargs)<func_end>
<docstring_start>
Return the

In [25]:
from langchain_core.prompts import PromptTemplate, PipelinePromptTemplate
from langchain.memory import ChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [26]:
prompt = PromptTemplate.from_template("What is the weather in {location}?")

In [27]:
status_template = """
car coordinates: lat:{lat}, lon:{lon}
current weather: {weather}
current temperature: {temperature}
current time: {time}
current date: {date}
destination: {destination}
"""
status_prompt = PromptTemplate.from_template(status_template)

In [28]:
status_prompt

PromptTemplate(input_variables=['date', 'destination', 'lat', 'lon', 'temperature', 'time', 'weather'], template='\ncar coordinates: lat:{lat}, lon:{lon}\ncurrent weather: {weather}\ncurrent temperature: {temperature}\ncurrent time: {time}\ncurrent date: {date}\ndestination: {destination}\n')

In [29]:
prompt = SystemMessage(content="You are a nice pirate")

In [30]:
full_template = """
<human>

{skills}

Current status of the vehicle:
{status}

User Query: Question: {question}


Please pick a function from the above options that best answers the user query and fill in the appropriate arguments.<human_end>
"""
full_prompt = PromptTemplate.from_template(full_template)

In [31]:
question_prompt = HumanMessage(content="How is the weather in Paris?")
input_prompts = [
    # ("skills", skills.SKILLS_PROMPT),
    ("status", status_prompt),
    # ("question", question_prompt),
]

In [32]:
pipeline_prompt = PipelinePromptTemplate(
  final_prompt=full_prompt, 
  pipeline_prompts=input_prompts
)

In [33]:
pipeline_prompt

PipelinePromptTemplate(input_variables=['date', 'time', 'weather', 'lon', 'lat', 'destination', 'temperature'], final_prompt=PromptTemplate(input_variables=['question', 'skills', 'status'], template='\n<human>\n\n{skills}\n\nCurrent status of the vehicle:\n{status}\n\nUser Query: Question: {question}\n\n\nPlease pick a function from the above options that best answers the user query and fill in the appropriate arguments.<human_end>\n'), pipeline_prompts=[('status', PromptTemplate(input_variables=['date', 'destination', 'lat', 'lon', 'temperature', 'time', 'weather'], template='\ncar coordinates: lat:{lat}, lon:{lon}\ncurrent weather: {weather}\ncurrent temperature: {temperature}\ncurrent time: {time}\ncurrent date: {date}\ndestination: {destination}\n'))])

In [34]:
history = ChatMessageHistory()
history.add_message(question_prompt)

In [35]:
history

ChatMessageHistory(messages=[HumanMessage(content='How is the weather in Paris?')])

In [36]:
history.add_ai_message(pipeline_prompt.format(
    date=datetime.now().strftime("%Y-%m-%d"),
    time=datetime.now().strftime("%H:%M:%S"),
    weather="sunny",
    temperature="20°C",
    destination="Paris",
    lat="48.8566",
    lon="2.3522",
    question="How is the weather in Paris?",
    # status=status,
    skills=skills.SKILLS_PROMPT,
    # system=system,
    # query=question,
  ))

In [37]:
history.messages[-1]

AIMessage(content="\n<human>\n\nOPTION:\n<func_start>get_weather(location: str = '', **kwargs)<func_end>\n<docstring_start>\nReturns the CURRENT weather in a specified location.\nArgs:\nlocation (string) : Required. The name of the location, could be a city or lat/longitude in the following format latitude,longitude (example: 37.7749,-122.4194).\n<docstring_end>\nOPTION:\n<func_start>get_forecast(city_name: str = '', when=0, **kwargs)<func_end>\n<docstring_start>\nReturns the weather forecast in a specified number of days for a specified city .\nArgs:\ncity_name (string) : Required. The name of the city.\nwhen (int) : Required. in number of days (until the day for which we want to know the forecast) (example: tomorrow is 1, in two days is 2, etc.)\n<docstring_end>\nOPTION:\n<func_start>find_route(lat_depart='0', lon_depart='0', city_depart='', address_destination='', depart_time='', **kwargs)<func_end>\n<docstring_start>\nReturn the distance and the estimated time to go to a specific d

In [38]:
print(
  pipeline_prompt.format(
    date=datetime.now().strftime("%Y-%m-%d"),
    time=datetime.now().strftime("%H:%M:%S"),
    weather="sunny",
    temperature="20°C",
    destination="Paris",
    lat="48.8566",
    lon="2.3522",
    question="How is the weather in Paris?",
    # status=status,
    skills=skills.SKILLS_PROMPT,
    # system=system,
    # query=question,
  )
)


<human>

OPTION:
<func_start>get_weather(location: str = '', **kwargs)<func_end>
<docstring_start>
Returns the CURRENT weather in a specified location.
Args:
location (string) : Required. The name of the location, could be a city or lat/longitude in the following format latitude,longitude (example: 37.7749,-122.4194).
<docstring_end>
OPTION:
<func_start>get_forecast(city_name: str = '', when=0, **kwargs)<func_end>
<docstring_start>
Returns the weather forecast in a specified number of days for a specified city .
Args:
city_name (string) : Required. The name of the city.
when (int) : Required. in number of days (until the day for which we want to know the forecast) (example: tomorrow is 1, in two days is 2, etc.)
<docstring_end>
OPTION:
<func_start>find_route(lat_depart='0', lon_depart='0', city_depart='', address_destination='', depart_time='', **kwargs)<func_end>
<docstring_start>
Return the distance and the estimated time to go to a specific destination from the current place, at a 

In [39]:
from langchain_community.llms import Ollama

llm = Ollama(model="nexusraven", stop=["\nReflection:", "\nThought:"])

In [40]:
chain = pipeline_prompt | llm

In [41]:
q = dict(
    date=datetime.now().strftime("%Y-%m-%d"),
    time=datetime.now().strftime("%H:%M:%S"),
    weather="sunny",
    temperature="20°C",
    destination="Paris",
    lat="48.8566",
    lon="2.3522",
    question="How is the weather in Paris?",
    # status=status,
    skills=skills.SKILLS_PROMPT,
    # system=system,
    # query=question,
  )

In [45]:
out_1 = chain.invoke(q)

In [46]:
func_out_1 = execute_function_call(out_1)

http://api.weatherapi.com/v1/current.json?key=d1c4d0d8ef6847339a0125737240903&q=Paris&aqi=no
http://api.weatherapi.com/v1/current.json?key=d1c4d0d8ef6847339a0125737240903&q=Paris&aqi=no


In [47]:
func_out_1

'The current weather in Paris, Ile-de-France, France is Light rain with a temperature of 17.0°C that feels like 17.0°C. Humidity is at 68%. Wind speed is 12.5 mph.'

In [1]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import create_self_ask_with_search_agent

# Test Langchain Agent

In [1]:
from langchain_core.prompts import PromptTemplate

In [2]:
import langchain
langchain.debug=False
from skills import get_weather, find_route, get_forecast, vehicle_status, vehicle

In [3]:
from typing import Union, List

from langchain.chains import LLMChain
from langchain.prompts import StringPromptTemplate
from langchain.tools.base import StructuredTool
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from skills import extract_func_args
from langchain.agents import (
    Tool,
    AgentExecutor,
    LLMSingleActionAgent,
    AgentOutputParser,
)

# https://github.com/nexusflowai/NexusRaven/blob/main/scripts/langchain_example.py
class RavenOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        print(f"llm_output: {llm_output}")
        if "Call: " in llm_output:
            # llm_output = llm_output.replace("Initial Answer: ", "Call: ")
            func_name, args = extract_func_args(llm_output)
            return AgentAction(
                tool=func_name,
                tool_input=args,
                log=f"Calling {func_name} with args: {args}",
            )

        # Check if agent should finish
        # if "Initial Answer:" in llm_output:
        #     return AgentFinish(
        #         return_values={
        #             "output": llm_output.strip()
        #             .split("\n")[1]
        #             .replace("Initial Answer: ", "")
        #             .strip()
        #         },
        #         log=llm_output,
        #     )
        
        return AgentFinish(
                return_values={
                    "output": llm_output.strip()
                },
                log=llm_output,
        )
        # else:
        #    raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")


In [4]:
tools = [
    StructuredTool.from_function(get_weather),
    StructuredTool.from_function(find_route),
    StructuredTool.from_function(vehicle_status),
]

In [5]:
import json

In [16]:
RAVEN_PROMPT_FUNC = """You are a helpful AI assistant in a car (vehicle), that follows instructions extremely well. You are aware of car's location through status information and take that into account before responding. Answer questions concisely and do not mention what you base your reply on."

{raven_tools}

Previous conversation history:
{history}

User Query: Question: {input}<human_end>
"""

RAVEN_PROMPT_ANS = """
<human>
You are a useful AI assistant in a car, that follows instructions extremely well. Help as much as you can. Answer questions concisely and do not mention what you base your reply on.

Previous conversation history:
{history}

Observation:{observation}

User Query: Question: {input}

Please respond in natural language. Do not refer to the provided context and observation in your response.<human_end>

Answer: """


STATUS_TEMPLATE = """We are at these geo coordinates: lat:{lat}, lon:{lon}, current time: {time}, current date: {date} and our destination is: {destination}
"""
status_prompt = PromptTemplate.from_template(STATUS_TEMPLATE)

class RavenPromptTemplate(StringPromptTemplate):
    template_func: str
    template_ans: str
    status_prompt: PromptTemplate
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        print(f"kwargs: {kwargs}")
        intermediate_steps = kwargs.pop("intermediate_steps")
        # if "input" in kwargs and type(kwargs["input"]) == dict:
        #     inputs = kwargs["input"]
        #     kwargs["status"] = inputs["status"]
        #     kwargs["input"] = inputs["input"]

        if intermediate_steps:
            step = intermediate_steps[-1]
            prompt = json.dumps(step[1][1], indent=4)
            kwargs["observation"] = prompt
            pp = self.template_ans.format(**kwargs).replace("{{", "{").replace("}}", "}")
            return pp

        # if "status" in kwargs:
        #     kwargs["status"] = self.status_prompt.format(**kwargs["status"])

        prompt = "<human>:\n"
        for tool in self.tools:
            func_signature, func_docstring = tool.description.split(" - ", 1)
            prompt += f'Function:\n<func_start>def {func_signature}<func_end>\n<docstring_start>\n"""\n{func_docstring}\n"""\n<docstring_end>\n'
        kwargs["raven_tools"] = prompt
        pp = self.template_func.format(**kwargs).replace("{{", "{").replace("}}", "}")
        return pp


raven_prompt = RavenPromptTemplate(
    template_func=RAVEN_PROMPT_FUNC,
    template_ans=RAVEN_PROMPT_ANS,
    status_prompt=status_prompt,
    tools=tools,
    input_variables=["input", "history", "intermediate_steps"],
)

In [17]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=2, input_key="input")

In [18]:
from langchain_community.llms import Ollama

# llm = Ollama(model="new-nexus")
llm = Ollama(model="nexusraven")
output_parser = RavenOutputParser()
llm_chain = LLMChain(llm=llm, prompt=raven_prompt)
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nReflection:", "\nThought:"],
    # stop=["\nReflection:"],
    allowed_tools=tools,
)

In [19]:
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, memory=memory, verbose=True)

In [1]:
agent_chain

NameError: name 'agent_chain' is not defined

In [21]:
from datetime import datetime

status = dict(
    date=datetime.now().strftime("%Y-%m-%d"),
    time=datetime.now().strftime("%H:%M:%S"),
    # temperature="20°C",
    destination="Kirchberg, Luxembourg",
    lat="48.8566",
    lon="2.3522",
)

In [22]:
call = agent_chain.run(
    input="What's our current location?",
    # status= status
)



> Entering new AgentExecutor chain...
kwargs: {'input': "What's our current location?", 'history': '', 'intermediate_steps': []}
llm_output: Call: vehicle_status(prompt='Question: What is our current location?') 
Calling vehicle_status with args: {'prompt': 'Question: What is our current location?'}

Reflection:('We are at Mondorf-les-Bains, Luxembourg, current time: 08:00:20, current date: 2025-03-29 and our destination is: Kirchberg Campus, Kirchberg.\n', {'location': 'Mondorf-les-Bains, Luxembourg', 'date': '2025-03-29', 'time': '08:00:20', 'destination': 'Kirchberg Campus, Kirchberg'})
kwargs: {'input': "What's our current location?", 'history': '', 'intermediate_steps': [(AgentAction(tool='vehicle_status', tool_input={'prompt': 'Question: What is our current location?'}, log="Calling vehicle_status with args: {'prompt': 'Question: What is our current location?'}"), ('We are at Mondorf-les-Bains, Luxembourg, current time: 08:00:20, current date: 2025-03-29 and our destination is:

In [23]:
vehicle.vehicle.location = "Mondorf-les-Bains, Luxembourg"

In [93]:
call

'The current weather conditions in Tokyo are partly cloudy, with a temperature of 17 degrees Celsius and a wind speed of 17.4 miles per hour.'

In [57]:
call = agent_chain.run(
        input="How about in Tokyo?",
        status= status
)



> Entering new AgentExecutor chain...
kwargs: {'input': 'How about in Tokyo?', 'status': {'date': '2024-05-02', 'time': '15:35:33', 'destination': 'Kirchberg, Luxembourg', 'lat': '48.8566', 'lon': '2.3522'}, 'history': 'Human: How is the weather in Luxembourg?\nAI: The weather in Luxembourg is moderate or heavy rain with thunder, with a temperature of 15 degrees Celsius. The wind is blowing from the west at a speed of 28.1 kilometers per hour, and there is a 75% chance of cloud cover. It feels like 13.7 degrees Celsius outside.', 'intermediate_steps': []}
You are a useful AI assistant in a car, that follows instructions extremely well. Help as much as you can. Answer questions concisely and do not mention what you base your reply on.
<human>:
\Function:
<func_start>def get_weather(location: str = '', **kwargs)<func_end>
<docstring_start>
"""
Returns the CURRENT weather in a specified location.
    Args:
    location (string) : Required. The name of the location, could be a city or la

In [67]:
call = agent_chain.run(
        input="What's our destination?",
        status= status
)



> Entering new AgentExecutor chain...
kwargs: {'input': "What's our destination?", 'status': {'date': '2024-05-02', 'time': '15:53:47', 'destination': 'Kirchberg, Luxembourg', 'lat': '48.8566', 'lon': '2.3522'}, 'history': '', 'intermediate_steps': []}
llm_output: Call: find_route(city_depart='Kirchberg', address_destination='Luxembourg') 
Calling find_route with args: {'city_depart': 'Kirchberg', 'address_destination': 'Luxembourg'}Luxembourg


NameError: name 'check_city_coordinates' is not defined

In [58]:
1

1

In [ ]:
call = agent_chain.run(
    "How is the weather in Tokyo?"
)



> Entering new AgentExecutor chain...
kwargs: {'input': 'How is the weather in Tokyo?', 'intermediate_steps': []}


KeyError: 'history'

In [80]:
call

'The current weather conditions in Tokyo, Japan are partly cloudy with temperatures of 17°C and a wind speed of 17.4 mph.'

In [84]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=2)

In [95]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)


In [96]:
agent_executor.run("How is the weather in Luxembourg?")



> Entering new AgentExecutor chain...
kwargs: {'input': 'How is the weather in Luxembourg?', 'intermediate_steps': []}


KeyError: 'history'

In [87]:
agent_executor.run("How about in Dubai?")



> Entering new AgentExecutor chain...
kwargs: {'input': 'How about in Dubai?', 'intermediate_steps': []}
llm_output:  
Call: find_route(city_depart='Dubai', address_destination='', depart_time=) 


SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

demo_ephemeral_chat_history_for_chain = ChatMessageHistory()
conversational_agent_executor = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: demo_ephemeral_chat_history_for_chain,
    input_messages_key="input",
    output_messages_key="output",
    history_messages_key="chat_history",
)

In [39]:

##############################
# Step 3: Construct Prompt ###
##############################


def construct_prompt(user_query: str, context):
    formatted_prompt = format_functions_for_prompt(get_weather, find_points_of_interest, find_route, get_forecast, search_along_route)
    formatted_prompt += f'\n\nContext : {context}'
    formatted_prompt += f"\n\nUser Query: Question: {user_query}\n"

    prompt = (
        "<human>:\n"
        + formatted_prompt
        + "Please pick a function from the above options that best answers the user query and fill in the appropriate arguments.<human_end>"
    )
    return prompt



In [40]:
# convert bytes to megabytes
def get_cuda_usage(): return round(torch.cuda.memory_allocated("cuda:0")/1024/1024,2)

In [ ]:
# might be deleted
# Compute a Simple equation
print(f"before everything: {get_cuda_usage()}")
prompt = construct_prompt("What restaurants are there on the road from Luxembourg Gare, which coordinates are lat 49.5999681, lon 6.1342493, to Thionville?", "")
print(f"after creating prompt: {get_cuda_usage()}")
model_output = pipe(
    prompt, do_sample=False, max_new_tokens=300, return_full_text=False
    )
print(model_output[0]["generated_text"])
#execute_function_call(pipe(construct_prompt("Is it raining in Belval, ?"), do_sample=False, max_new_tokens=300, return_full_text=False))

In [ ]:
print(f"creating the pipe of model output: {get_cuda_usage()}")
result = execute_function_call(model_output)
print(f"after execute function call: {get_cuda_usage()}")
del model_output
import gc         # garbage collect library
gc.collect()
torch.cuda.empty_cache() 
print(f"after garbage collect and empty_cache: {get_cuda_usage()}")
#print("Model Output:", model_output)
# print("Execution Result:", result)

## functions to process the anwser and the question

In [ ]:
#generation of text with Stable beluga 
def gen(p, maxlen=15, sample=True):
    toks = tokr(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
    return tokr.batch_decode(res)

#to have a prompt corresponding to the specific format required by the fine-tuned model Stable Beluga
def mk_prompt(user, syst="### System:\nYou are a useful AI assistant in a car, that follows instructions extremely well. Help as much as you can. Answer questions concisely and do not mention what you base your reply on.\n\n"): return f"{syst}### User: {user}\n\n### Assistant:\n"

In [ ]:
def car_answer_only(complete_answer, general_context):
    """returns only the AI assistant answer, without all context, to reply to the user"""
    pattern = r"Assistant:\\n(.*)(</s>|[.!?](\s|$))" #pattern = r"Assistant:\\n(.*?)</s>"

    match = re.search(pattern, complete_answer, re.DOTALL)

    if match:
        # Extracting the text
        model_answer = match.group(1)
        #print(complete_answer)
    else:
        #print(complete_answer)
        model_answer = "There has been an error with the generated response." 

    general_context +=  model_answer
    return (model_answer, general_context)
#print(model_answer)

In [ ]:
def FnAnswer(general_context, ques, place, time, delete_history, state):
    """function to manage the two different llms (function calling and basic answer) and call them one after the other"""
    # Initialize state if it is None
    if delete_history == "Yes":
        state = None
    if state is None:
        conv_context = []
        conv_context.append(general_context)
        state = {}
        state['context'] = conv_context
        state['number'] = 0
        state['last_question'] = ""
        
    if type(ques) != str: 
        ques = ques[0]
        
    place = definePlace(place) #which on the predefined places it is
    
    formatted_context = '\n'.join(state['context'])
        
    #updated at every question
    general_context = f"""
    Recent conversation history: '{formatted_context}' (If empty, this indicates the beginning of the conversation).

    Previous question from the user: '{state['last_question']}' (This may or may not be related to the current question).

    User information: The user is inside a car in {place[0]}, with latitude {place[1]} and longitude {place[2]}. The user is mobile and can drive to different destinations. It is currently {time}

    """
    #first llm call (function calling model, NexusRaven)
    model_output= pipe(construct_prompt(ques, general_context), do_sample=False, max_new_tokens=300, return_full_text=False)
    call = execute_function_call(model_output) #call variable is formatted to as a call to a specific function with the required parameters
    print(call)
    #this is what will erase the model_output from the GPU memory to free up space
    del model_output
    import gc         # garbage collect library
    gc.collect()
    torch.cuda.empty_cache() 
        
    #updated at every question
    general_context += f'This information might be of help, use if it seems relevant, and ignore if not relevant to reply to the user: "{call}". '
    
    #question formatted for the StableBeluga llm (second llm), using the output of the first llm as context in general_context
    question=f"""Reply to the user and answer any question with the help of the provided context.

    ## Context

    {general_context} .

    ## Question

    {ques}"""

    complete_answer = str(gen(mk_prompt(question), 100)) #answer generation with StableBeluga (2nd llm)

    model_answer, general_context= car_answer_only(complete_answer, general_context) #to retrieve only the car answer 
    
    language = pipe_language(model_answer, top_k=1, truncation=True)[0]['label'] #detect the language of the answer, to modify the text-to-speech consequently
    
    state['last_question'] = ques #add the current question as 'last question' for the next question's context
    
    state['number']= state['number'] + 1  #adds 1 to the number of interactions with the car

    state['context'].append(str(state['number']) + '. User question: '+ ques + ', Model answer: ' + model_answer) #modifies the context
    
    #print("contexte : " + '\n'.join(state['context']))
    
    if len(state['context'])>5: #6 questions maximum in the context to avoid having too many information
        state['context'] = state['context'][1:]

    return model_answer, state['context'], state, language

In [ ]:
def transcript(general_context, link_to_audio, voice, place, time, delete_history, state):
    """this function manages speech-to-text to input Fnanswer function and text-to-speech with the Fnanswer output"""
    # load audio from a specific path
    audio_path = link_to_audio
    audio_array, sampling_rate = librosa.load(link_to_audio, sr=16000)  # "sr=16000" ensures that the sampling rate is as required


    # process the audio array
    input_features = processor(audio_array, sampling_rate, return_tensors="pt").input_features


    predicted_ids = modelw.generate(input_features)

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    quest_processing = FnAnswer(general_context, transcription, place, time, delete_history, state)
    
    state=quest_processing[2]
    
    print("langue " + quest_processing[3])

    tts.tts_to_file(text= str(quest_processing[0]),
                file_path="output.wav",
                speaker_wav=f'Audio_Files/{voice}.wav',
                language=quest_processing[3],
                emotion = "angry")

    audio_path = "output.wav"
    return audio_path, state['context'], state

In [ ]:
def definePlace(place):
    if(place == 'Luxembourg Gare, Luxembourg'):
        return('Luxembourg Gare', '49.5999681', '6.1342493' )
    elif (place =='Kirchberg Campus, Kirchberg'):
        return('Kirchberg Campus, Luxembourg', '49.62571206478235', '6.160082636815114')
    elif (place =='Belval Campus, Belval'):
        return('Belval-Université, Esch-sur-Alzette', '49.499531', '5.9462903')
    elif (place =='Eiffel Tower, Paris'):
        return('Eiffel Tower, Paris', '48.8582599', '2.2945006')
    elif (place=='Thionville, France'):
        return('Thionville, France', '49.357927', '6.167587')

## Interfaces (text and audio)

In [ ]:
#INTERFACE WITH ONLY TEXT

# Generate options for hours (00-23) 
hour_options = [f"{i:02d}:00:00" for i in range(24)]

model_answer= ''
general_context= ''
# Define the initial state with some initial context.
print(general_context)
initial_state = {'context': general_context}
initial_context= initial_state['context']
# Create the Gradio interface.
iface = gr.Interface(
    fn=FnAnswer,
    inputs=[
        gr.Textbox(value=initial_context, visible=False),
        gr.Textbox(lines=2, placeholder="Type your message here..."),
        gr.Radio(choices=['Luxembourg Gare, Luxembourg', 'Kirchberg Campus, Kirchberg', 'Belval Campus, Belval', 'Eiffel Tower, Paris', 'Thionville, France'], label='Choose a location for your car', value= 'Kirchberg Campus, Kirchberg', show_label=True),
        gr.Dropdown(choices=hour_options, label="What time is it?", value = "08:00:00"),
        gr.Radio(["Yes", "No"], label="Delete the conversation history?", value = 'No'),
        gr.State()  # This will keep track of the context state across interactions.
    ],
    outputs=[
        gr.Textbox(),
        gr.Textbox(visible=False),
        gr.State()
    ]
)
gr.close_all()
# Launch the interface.
iface.launch(debug=True, share=True, server_name="0.0.0.0", server_port=7860)
#contextual=gr.Textbox(value=general_context, visible=False)
#demo = gr.Interface(fn=FnAnswer, inputs=[contextual,"text"], outputs=["text", contextual])

#demo.launch()

## Other possible APIs to use

In [ ]:

def search_nearby(lat, lon, city, key):
    """
    :param lat: latitude
    :param lon: longitude
    :param key: api key
    :param type: type of poi
    :return: [5] results ['poi']['name']/['freeformAddress'] || ['position']['lat']/['lon']
    """
    results = []

    r = requests.get('https://api.tomtom.com/search/2/nearbySearch/.json?key={0}&lat={1}&lon={2}&radius=10000&limit=50'.format(
                        key,
                        lat,
                        lon
    ))

    for result in r.json()['results']:
        results.append(f"The {' '.join(result['poi']['categories'])} {result['poi']['name']} is {int(result['dist'])} meters far from {city}")
        if len(results) == 7:
            break

    return ". ".join(results)


print(search_nearby('49.625892805337514', '6.160417066963513', 'your location', TOMTOM_KEY))